In [8]:
from utils import set_env
import time
set_env()
import pandas as pd
from datasets.arrow_dataset import Dataset
import pyarrow as pa
i1=time.time()
from transformers import AutoTokenizer, AutoModelForCausalLM
i2=time.time()
print('1', i2-i1)
from datasets import load_dataset
i3=time.time()
print('2',i3-i2)
import torch
i4=time.time()
print('3', i4-i3)

from trl import SFTTrainer
i5=time.time()
print('4',i5-i4)
from peft import LoraConfig, PeftModel
from trl import SFTConfig
from tqdm import trange

1 3.743171691894531e-05
2 0.00032806396484375
3 0.00011682510375976562
4 0.00010776519775390625


In [9]:
base_model_id = "meta-llama/Llama-3.2-1B"

model_original = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_eos_token=True,
    add_bos_token=True, 
)
tokenizer.pad_token = tokenizer.eos_token

In [10]:
DATA_PATH='Data'
DATASET='mami'
# dataset_name = "scooterman/guanaco-llama3-1k"
train, test= torch.load(f'{DATA_PATH}/{DATASET}_trainset_llm.pkl', weights_only=True), torch.load(f'{DATA_PATH}/{DATASET}_testset_llm.pkl', weights_only=True)
for i in range(len(train)):
    train[i]['prompt_all_text']=f"{train[i]['prompt_all_text']} {'good' if train[i]['label']==0 else 'bad'}"
train=Dataset(pa.Table.from_pylist([{'text':x['prompt_all_text']} for x in train]))
# test=Dataset(pa.Table.from_pylist([{'text':x['prompt_all_text']} for x in test]))

In [11]:
print(train[0]['text'])

Example 1: Milk Milkzip. Context: A pitcher of milk and a piece of cheese. The person in the image is a christian. Entity: Archive milk zip software lossless compression skimmed milk powdered milk cheese. Demographic: Unk. Classification: good

Example 2: Nappcom Perfect Women Starts undressing while you play video games MemeCentercom. Context: A woman in a lingerie sitting on a couch. A white woman in a lingerie posing on a couch in front of a tv screen with a . A woman is sitting on a couch in a lingerie and a t-shirt with a picture of a . Switzerland is the country that the person in the image comes from switzerland is the country that the person in the image. A woman in a lingerie posing on a couch. Entity: Breast photo shoot science geometry violet adipose tissue sternum physics surgical operation mammary gland line thorax brand light skin prosthesis mathematics screenshot font breast augmentation. Demographic: White female white female. Classification: bad

Question: HOW YOU SIT 

In [16]:
print(test[0]['prompt_all_text'])

Example 1: FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMAN JOINS THE GROUP imgflipcom. Context: A group of people posing for a picture. He is a white man with a blond wig and a blond wig and a blond . A man and a woman are posing in front of a mirror. Switzerland - switzerland - switzerland - switzerland - s. He is a christian, he is a muslim, he is a jew, he is. Entity: Sitcom kaley cuoco sheldon cooper bill prady johnny galecki jim parsons the big bang theory penny chuck lorre. Demographic: White male indian male white male white female white male indian male. Classification: good

Example 2: Normal gamers use this Pro gamers use this But only LEGENDS use this MemeCentercom. Context: A picture of a chair and a picture of a chair and a picture of a chair and a picture of a. The person in the image is a christian agnostic agnostic agnostic agno. Entity: Design gaming chair legend gamer good create. Demographic: Unk. Classification: good

Question: Access USER USER Jenner ate grilled ch

In [ ]:
# Output directory where the results and checkpoint are stored
import copy


output_dir = "./results"
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 100
logging_steps = 5
training_arguments = SFTConfig(
    max_seq_length=1024,
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=2,
    optim=optim,
    save_steps=100,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=True,
    max_grad_norm=max_grad_norm,
    max_steps=-1,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    dataset_text_field="text",
    report_to="none"
)
model=copy.deepcopy(model_original)
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.train()

In [5]:
from transformers import pipeline
pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer,
    stop_strings=["\n",'.'],max_new_tokens=2000)

In [19]:
correct=0
res1=[]
pbar=trange(len(test))
for i in pbar:
    res=pipe(test[i]['prompt_all_text'], tokenizer=tokenizer)[0]['generated_text']
    res_1=res[len(test[i]['prompt_all_text']):].strip()
    res1.append(res_1)
    guess=0 if res_1=='good' else 1
    correct+=guess==test[i]['label']
    pbar.set_postfix_str(f"Acc:{correct/(i+1):.4f}")
print(pd.Series(res1).apply(lambda x:x.strip()).value_counts())
print(f"Acc:{correct/len(test):.4f}")

 23%|██▎       | 231/995 [00:18<00:59, 12.77it/s, Acc:1.0000]


KeyboardInterrupt: 

In [30]:
i=1
res=pipe(test[i]['prompt_all_text'], tokenizer=tokenizer)[0]['generated_text']

print(res[len(test[i]['prompt_all_text']):].strip())
test[1]['label']

bad


tensor(1)